In [37]:
import torch
import os
from torchvision import datasets, models, transforms
from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import glob
import copy
import neptune
import matplotlib.gridspec as gridspec
cudnn.benchmark = True
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# Models with 10 classes

# ResNet18

In [38]:
data_transforms = {
    'train': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((256,256)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'E:\datasets\dataset'

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32,
                                             shuffle=True, num_workers=8)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes
device = torch.device("cpu")

In [39]:
model_ft = models.resnet18()

num_ftrs = model_ft.fc.in_features

model_ft.fc = nn.Linear(num_ftrs, 10)
model_ft = model_ft.to(device)
model_ft.load_state_dict(torch.load('weights-resnet18-bestloss.pth',map_location ='cpu'))

<All keys matched successfully>

In [40]:
nb_classes = 10

predlist=torch.zeros(0,dtype=torch.long, device=device)
lbllist=torch.zeros(0,dtype=torch.long, device=device)

with torch.no_grad():
    for i, (inputs, classes) in enumerate(dataloaders['val']):
        inputs = inputs.to(device)
        classes = classes.to(device)
        outputs = model_ft(inputs)
        _, preds = torch.max(outputs, 1)
        predlist=torch.cat([predlist,preds.view(-1).cpu()])
        lbllist=torch.cat([lbllist,classes.view(-1).cpu()])

target_names = ['ясно' ,'облачно' ,'туман' ,'мороз' ,'град' ,'молния' , 'дождь', 'радуга',
           'снег', 'восход']
print(classification_report(lbllist.numpy(), predlist.numpy(),target_names=target_names))

              precision    recall  f1-score   support

        ясно       0.83      0.82      0.82        99
     облачно       0.83      0.83      0.83       129
       туман       0.89      0.90      0.90       147
       мороз       0.91      0.93      0.92       165
        град       0.91      0.97      0.94        64
      молния       1.00      0.98      0.99        43
       дождь       0.89      0.88      0.88       177
      радуга       1.00      1.00      1.00        28
        снег       0.84      0.81      0.83       162
      восход       0.99      0.99      0.99        69

    accuracy                           0.89      1083
   macro avg       0.91      0.91      0.91      1083
weighted avg       0.89      0.89      0.89      1083



## ResNet34

In [41]:
data_transforms = {
    'train': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((256,256)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'E:\datasets\dataset'

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32,
                                             shuffle=True, num_workers=8)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes
device = torch.device("cpu")

In [42]:
model_ft = models.resnet34()

num_ftrs = model_ft.fc.in_features

model_ft.fc = nn.Linear(num_ftrs, 10)
model_ft = model_ft.to(device)
model_ft.load_state_dict(torch.load('weights-resnet34-bestloss.pth',map_location ='cpu'))

<All keys matched successfully>

In [43]:
with torch.no_grad():
    for i, (inputs, classes) in enumerate(dataloaders['val']):
        inputs = inputs.to(device)
        classes = classes.to(device)
        outputs = model_ft(inputs)
        _, preds = torch.max(outputs, 1)
        predlist=torch.cat([predlist,preds.view(-1).cpu()])
        lbllist=torch.cat([lbllist,classes.view(-1).cpu()])

target_names = ['ясно' ,'облачно' ,'туман' ,'мороз' ,'град' ,'молния' , 'дождь', 'радуга',
           'снег', 'восход']
print(classification_report(lbllist.numpy(), predlist.numpy(),target_names=target_names))

              precision    recall  f1-score   support

        ясно       0.85      0.84      0.85       198
     облачно       0.84      0.85      0.84       258
       туман       0.90      0.93      0.92       294
       мороз       0.90      0.93      0.92       330
        град       0.95      0.97      0.96       128
      молния       1.00      0.98      0.99        86
       дождь       0.88      0.88      0.88       354
      радуга       1.00      0.98      0.99        56
        снег       0.86      0.81      0.84       324
      восход       0.99      0.99      0.99       138

    accuracy                           0.90      2166
   macro avg       0.92      0.92      0.92      2166
weighted avg       0.90      0.90      0.90      2166



## ResNet152

In [44]:
data_transforms = {
    'train': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((256,256)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'E:\datasets\dataset'

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=10,
                                             shuffle=True, num_workers=0)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes
device = torch.device("cpu")

In [45]:
model_ft = models.resnet152()

num_ftrs = model_ft.fc.in_features

model_ft.fc = nn.Linear(num_ftrs, 10)
model_ft = model_ft.to(device)
model_ft.load_state_dict(torch.load('weights-resnet152-bestloss.pth',map_location ='cpu'))

<All keys matched successfully>

In [46]:
with torch.no_grad():
    for i, (inputs, classes) in enumerate(dataloaders['val']):
        inputs = inputs.to(device)
        classes = classes.to(device)
        outputs = model_ft(inputs)
        _, preds = torch.max(outputs, 1)
        predlist=torch.cat([predlist,preds.view(-1).cpu()])
        lbllist=torch.cat([lbllist,classes.view(-1).cpu()])

target_names = ['ясно' ,'облачно' ,'туман' ,'мороз' ,'град' ,'молния' , 'дождь', 'радуга',
           'снег', 'восход']
print(classification_report(lbllist.numpy(), predlist.numpy(),target_names=target_names))

              precision    recall  f1-score   support

        ясно       0.84      0.84      0.84       297
     облачно       0.81      0.85      0.83       387
       туман       0.91      0.92      0.91       441
       мороз       0.90      0.93      0.91       495
        град       0.92      0.95      0.94       192
      молния       1.00      0.98      0.99       129
       дождь       0.87      0.88      0.87       531
      радуга       1.00      0.96      0.98        84
        снег       0.85      0.78      0.82       486
      восход       0.98      0.98      0.98       207

    accuracy                           0.89      3249
   macro avg       0.91      0.91      0.91      3249
weighted avg       0.89      0.89      0.89      3249



## Efinetb4

In [47]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((128 , 128)),   
        transforms.RandomRotation(10),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((128 , 128)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'E:\datasets\dataset'

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
              for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=16,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) 
              for x in ['train', 'val']}

class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [48]:
model_ft = models.efficientnet_b4()
num_ftrs = model_ft.classifier[1].in_features
model_ft.fc = nn.Linear(num_ftrs, 10)
model_ft = model_ft.to(device)
model_ft.load_state_dict(torch.load('weights-efinetb4-bestloss.pth',map_location ='cpu'))

<All keys matched successfully>

In [49]:
with torch.no_grad():
    for i, (inputs, classes) in enumerate(dataloaders['val']):
        inputs = inputs.to(device)
        classes = classes.to(device)
        outputs = model_ft(inputs)
        _, preds = torch.max(outputs, 1)
        predlist=torch.cat([predlist,preds.view(-1).cpu()])
        lbllist=torch.cat([lbllist,classes.view(-1).cpu()])

target_names = ['ясно' ,'облачно' ,'туман' ,'мороз' ,'град' ,'молния' , 'дождь', 'радуга',
           'снег', 'восход']
print(classification_report(lbllist.numpy(), predlist.numpy(),target_names=target_names))

              precision    recall  f1-score   support

        ясно       0.84      0.84      0.84       396
     облачно       0.80      0.84      0.82       516
       туман       0.91      0.92      0.91       588
       мороз       0.90      0.92      0.91       660
        град       0.93      0.95      0.94       256
      молния       0.99      0.98      0.99       172
       дождь       0.87      0.87      0.87       708
      радуга       1.00      0.96      0.98       112
        снег       0.84      0.78      0.81       648
      восход       0.98      0.98      0.98       276

    accuracy                           0.88      4332
   macro avg       0.91      0.90      0.90      4332
weighted avg       0.88      0.88      0.88      4332



# Models with 11 classes

## Efinetb4

In [51]:
data_transforms = {
    'test': transforms.Compose([
        transforms.Resize((128 , 128)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'E:\datasets\dataset_noweather'

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ["test"]}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=16,
                                             shuffle=True, num_workers=4)
              for x in ["test"]}
dataset_sizes = {x: len(image_datasets[x]) for x in ["test"]}
class_names = image_datasets["test"].classes
device = torch.device("cpu")

In [52]:
model_ft = models.efficientnet_b4()
num_ftrs = model_ft.classifier[1].in_features
model_ft.fc = nn.Linear(num_ftrs, 11)
model_ft = model_ft.to(device)
model_ft.load_state_dict(torch.load('weights-no_weather_class-efinetb4-bestloss.pth',map_location ='cpu'))

<All keys matched successfully>

In [54]:
nb_classes = 11

predlist=torch.zeros(0,dtype=torch.long, device=device)
lbllist=torch.zeros(0,dtype=torch.long, device=device)

with torch.no_grad():
    for i, (inputs, classes) in enumerate(dataloaders['test']):
        inputs = inputs.to(device)
        classes = classes.to(device)
        outputs = model_ft(inputs)
        _, preds = torch.max(outputs, 1)
        predlist=torch.cat([predlist,preds.view(-1).cpu()])
        lbllist=torch.cat([lbllist,classes.view(-1).cpu()])
        from sklearn.metrics import classification_report
target_names = ['ясно' ,'облачно' ,'туман' ,'мороз' ,'град' ,'молния', 'Нет погоды' , 'дождь', 'радуга',
           'снег', 'восход']
print(classification_report(lbllist.numpy(), predlist.numpy(),target_names=target_names))

              precision    recall  f1-score   support

        ясно       0.76      0.71      0.73       101
     облачно       0.74      0.76      0.75       130
       туман       0.90      0.91      0.90       148
       мороз       0.91      0.89      0.90       165
        град       0.89      0.95      0.92        65
      молния       0.98      1.00      0.99        44
  Нет погоды       0.93      0.90      0.91        82
       дождь       0.86      0.77      0.82       177
      радуга       0.93      0.93      0.93        30
        снег       0.77      0.84      0.80       163
      восход       0.93      0.97      0.95        71

    accuracy                           0.85      1176
   macro avg       0.87      0.88      0.87      1176
weighted avg       0.85      0.85      0.85      1176



# ResNet152

In [55]:
data_transforms = {
    'test': transforms.Compose([
        transforms.Resize((256 , 256)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'E:\datasets\dataset_noweather'

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ["test"]}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=10,
                                             shuffle=True, num_workers=2)
              for x in ["test"]}
dataset_sizes = {x: len(image_datasets[x]) for x in ["test"]}
class_names = image_datasets["test"].classes
device = torch.device("cpu")

In [58]:
model_ft = models.resnet152(pretrained=True) 
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, len(class_names))
model_ft = model_ft.to(device)
model_ft.load_state_dict(torch.load('weights-resnet152-bestloss-no_weather.pth',map_location ='cpu'))

<All keys matched successfully>

In [59]:
nb_classes = 11

predlist=torch.zeros(0,dtype=torch.long, device=device)
lbllist=torch.zeros(0,dtype=torch.long, device=device)

with torch.no_grad():
    for i, (inputs, classes) in enumerate(dataloaders['test']):
        inputs = inputs.to(device)
        classes = classes.to(device)
        outputs = model_ft(inputs)
        _, preds = torch.max(outputs, 1)
        predlist=torch.cat([predlist,preds.view(-1).cpu()])
        lbllist=torch.cat([lbllist,classes.view(-1).cpu()])
        from sklearn.metrics import classification_report
target_names = ['ясно' ,'облачно' ,'туман' ,'мороз' ,'град' ,'молния', 'Нет погоды' , 'дождь', 'радуга',
           'снег', 'восход']
print(classification_report(lbllist.numpy(), predlist.numpy(),target_names=target_names))

              precision    recall  f1-score   support

        ясно       0.79      0.82      0.81       101
     облачно       0.80      0.82      0.81       130
       туман       0.94      0.91      0.92       148
       мороз       0.88      0.88      0.88       165
        град       0.89      0.83      0.86        65
      молния       0.98      0.98      0.98        44
  Нет погоды       0.87      0.93      0.90        82
       дождь       0.83      0.83      0.83       177
      радуга       0.97      0.97      0.97        30
        снег       0.83      0.79      0.81       163
      восход       0.95      0.99      0.97        71

    accuracy                           0.87      1176
   macro avg       0.88      0.89      0.88      1176
weighted avg       0.87      0.87      0.87      1176

